# 유튜브 - 미래에셋 스마트 머니
필요한 칼럼  
재생목록, 업로드 날짜, 제목, 조회 수, 좋아요 수, 러닝 타임, 댓글 수

In [107]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import json
import time

In [108]:
# 모든 스크롤을 내리는 함수
def scrolling(driver):
    # 현재 페이지의 높이를 기록
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    start = time.time()
    while True:
        # 스크롤을 내린다.
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.5)
        
        # 새로운 페이지의 높이를 기록한다.
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        end = time.time()
        # 현재 스크롤의 높이와 이전 스크롤의 높이가 같으면 무한 스크롤이 끝났다고 판단하고 루프를 빠져나온다.
        if new_height == last_height or end - start > 6:
            break
        
        # 현재 높이를 새로운 높이로 변경한다.
        last_height = new_height

In [109]:
# 데이터프레임 초기화
youtube_data = pd.DataFrame(columns = ['playlist', 'date', 'title', 'num_review', 'num_good', 'num_views', 'video_url'])

# 재생목록 들어가고 난 후, 한 동영상에 대해 필요한 정보 크롤링
def video_crawling(driver):
    global youtube_data

    # 플레이리스트
    playlist_ = driver.find_element(By.XPATH, '//*[@id="header-description"]/h3[1]/yt-formatted-string/a').text

    # 더보기 클릭
    while True:
        try:
            driver.find_element(By.XPATH, '//*[@id="expand"]').click()
            content_ = driver.find_element(By.XPATH, '//*[@id="description-inline-expander"]').text
            break
        except:
            driver.execute_script(f"window.scrollTo(0, 150)")


    # 스크롤 올리고 동영상 한번 클릭
    driver.execute_script("window.scrollTo(0,{})".format(0))
    driver.find_element(By.XPATH, '//*[@id="movie_player"]/div[1]/video').click()
    time.sleep(2.5)

    title_, num_good_, views_, date_, current_url_ = '', '', '', '', ''
    # 각종 정보
    title_ = driver.find_element(By.XPATH, '//*[@id="title"]/h1/yt-formatted-string').text # 제목
    num_good_ = driver.find_element(By.XPATH, '//*[@id="top-level-buttons-computed"]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]').text # 좋아요 수
    views_ = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]').text # 조회 수
    date_ = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]').text # 날짜
    current_url_ = driver.current_url # url
    
#    running_time_ = driver.find_element(By.XPATH, '//*[@id="movie_player"]/div[35]/div[2]/div[1]/div[1]/span[2]/span[3]').text # 러닝타임

    # 스크롤 내리고 댓글
    number = 0
    num_review_ = ''
    while not num_review_:
        try:
            num_review_ = driver.find_element(By.XPATH, '//*[@id="count"]/yt-formatted-string').text
        except:
            if number > 5000:
                break
            number += 500
            driver.execute_script("window.scrollTo(0,{})".format(number))
            time.sleep(1)

    # 다시 스크롤 올리기
    driver.execute_script("window.scrollTo(0,{})".format(0))
    
    # 스크랩한 것들 전부 저장
    new_row = {'playlist' : playlist_,
               'date' : date_,
               'title' : title_,
               'num_review' : num_review_,
               'num_good' : num_good_,
               'num_views' : views_,
               'content' : content_,
               'video_url' : current_url_}
    
    # 데이터프레임에 업로드
    youtube_data = pd.concat([youtube_data, pd.DataFrame([new_row])])

    print(playlist_)
    print(title_)
    print(num_good_)
    print(views_)
    print(date_)
#    print(running_time_)
    print(num_review_)
    print(current_url_)

In [110]:
from selenium.webdriver.common.by import By

# 미래에셋 유튜브 가져오기
url = "https://www.youtube.com/@SmartMoney0/playlists"
driver = webdriver.Chrome()
driver.get(url)

# 모든 재생목록 받아두기
scrolling(driver)
title_elements = driver.find_elements(By.XPATH, '//*[@id="playlist-thumbnails"]/ytd-playlist-video-thumbnail-renderer/yt-image/img')

# 다시 스크롤 제일 위로 올려서 재생목록 하나씩 들어가기
driver.execute_script("window.scrollTo(0, 0);") # 스크롤 제일 위로
num_playlist = len(title_elements) # 총 81개 재생목록

scroll = 0
for idx in range(num_playlist):
    while True:
        # try:
        if idx >= 29:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            print('idx가 30을 넘었습니다')
            time.sleep(2)
            if idx >= 59: 
                driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
                print('idx가 59를 넘었습니다')
                time.sleep(2)
        if idx % 4 == 0:
            scroll += 225
        driver.execute_script(f"window.scrollTo(0, {scroll})")
        time.sleep(2)
            # 재생목록 들어가기
        if idx >= 1:
            print('새로운 재생목록으로 들어갈 준비')
            # 재생목록 다시 받아두자
            title_elements = driver.find_elements(By.XPATH, '//*[@id="playlist-thumbnails"]/ytd-playlist-video-thumbnail-renderer/yt-image/img')
        time.sleep(2)
        title_elements[idx].click() # 재생목록 들어가기
        print(f'{idx + 1}번째 재생목록 들어갔습니다\n')
        break
        # except Exception as e:
        #     # 오류나면 스크롤 20씩 내리기
        #     scroll += 20
        #     driver.execute_script(f"window.scrollTo(0, {scroll})")
        #     print(f'{e}. 재생목록 못들어가서 스크롤 내립니다\n')
    
    time.sleep(2)

    # 재생목록의 동영상 수
    video_list = driver.find_elements(By.XPATH, '//*[@id="playlist-items"]')
    for i in range(len(video_list)):
        video_list[i].click()
        time.sleep(1)
        video_crawling(driver)
        print('=========================')
    
    # 해당 재생목록 작업 끝나면 다시 기존의 url 재접속
    driver.get(url)
    time.sleep(2)

1번째 재생목록 들어갔습니다

슬기로운 투자 생각
[슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이? #중국증시 #로보락
498
조회수 11,489회
2024. 5. 20.
댓글 14개
https://www.youtube.com/watch?v=JrHgv0R8RIQ&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=1
슬기로운 투자 생각
[슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? #Hangseng #중국주식 #고배당주
319
조회수 5,052회
2024. 6. 17.
댓글 15개
https://www.youtube.com/watch?v=u1j5cB9Dqpc&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=2
슬기로운 투자 생각
반등 랠리 기대주 찾기 "주가하락, 나는 진짜 억울해" #CATL | 슬기로운 투자 생각
38
조회수 7,797회
2024. 6. 22.
댓글 6개
https://www.youtube.com/watch?v=Ms6p1HznC_U&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=3
슬기로운 투자 생각
🌝중국 주도주 반등 랠리, 제대로 시동겁니다. 🚌 놓치기 전에 타세요~ [슬기로운 투자 생각] ep.7 #중국주식 #주도주
278
조회수 5,181회
2024. 6. 26.
댓글 17개
https://www.youtube.com/watch?v=7Yoej1DUR_0&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=4
새로운 재생목록으로 들어갈 준비
2번째 재생목록 들어갔습니다

유토피아
[유토피아] AI는 Apple iphone의 줄임말? | 2024.5.17(금)
378
조회수 8,722회
2024. 5. 17.
댓글 14개
https://www.youtube.com/watch?v=GDHMzjCLSjk&list=PL1-YXgQy7nWx-kLXcB

KeyboardInterrupt: 

In [112]:
youtube_data.to_csv("youtube_data.csv")

In [113]:
youtube_data

,playlist,date,title,num_review,num_good,num_views,video_url,content
0,슬기로운 투자 생각,2024. 5. 20.,"[슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이? #중국증시 #로보락",댓글 14개,498,"조회수 11,489회",https://www.youtube.com/watch?v=JrHgv0R8RIQ&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=1,【이인구의 슬기로운 투자 생각】 ep4. 중국투자전략\n출연: 이인구 이사(미래에셋증권 디지털리서치팀)\n\n00:00 하이라이트\n00:25 중국 매크로 점검 \n03:35 중국 해외 무역 비중 변화\n05:21 중국 주요 가전업체 주가 \n06:20 클로징\n\n#중국증시 #투자전략 #이인구 #미래에셋증권 #로보락 #roborock \n#KOR #ENG #IND \n\n[▼보고서 보기▼]\n[중국 투자 전략] 경기 회복의 새로운 시그널\nhttps://securities.miraeasset.com/bbs...\n챕터\n모두 보기\n하이라이트\n0:00\n중국 매크로 점검\n0:25\n중국 해외 무역 비중 변화\n3:35\n중국 주요 가전업체 주가\n5:21\n스크립트\n스크립트를 보면서 시청하세요.\n스크립트 표시\n미래에셋 스마트머니\n구독자 149만명\n동영상\n정보\n페이스북\n[슬기로운 투자 생각] 지금이 진짜 중국 증시 저점일까?\n제작자: 미래에셋 스마트머니\n조회수 2.1만회 · 3개월 전\n간략히
0,슬기로운 투자 생각,2024. 6. 17.,"[슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? #Hangseng #중국주식 #고배당주",댓글 15개,319,"조회수 5,052회",https://www.youtube.com/watch?v=u1j5cB9Dqpc&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=2,"【이인구의 슬기로운 투자 생각】\nep5. 중국 Go! 배당 주!\n\n00:00 하이라이트\n00:49 중국 배당주 투자 열풍\n01:37 투자대안의 부재와 높은 배당수익률\n03:50 중국판 밸류업! 신국9조 \n05:08 홍콩배당주는 세제 혜택?\n06:35 배당+00은 더욱 중요!\n09:10 중국, 홍콩 증시의 새로운 투자 물결..고배당 투자\n09:50 클로징\n\n[슬기로운 투자 생각] ▼ 더보기\nep.1. 지금이 진짜 중국 증시 저점일까?\n • [슬기로운 투자 생각] 지금이 진짜 중국 증시 저점일까? \n\nep.2. 중국 증시 저점? 중국의 새로운 주도주는?\n • [슬기로운 투자 생각] 중국 증시 저점? 중국의 새로운 주도주는? \n\nep.3. 중국의 전략적 투자 방향?\n • [슬기로운 투자 생각] 중국의 전략적 투자 방향? \n\nep.4.중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이?\n • [슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는... \n\n\n[▼보고서 보기▼]\n[중국 투자 전략] 2024년, 중국 투자의 세가지 키워드\nhttps://securities.miraeasset.com/bbs...\n\n[중국 투자 전략] 2024년, 환승 차이나\nhttps://securities.miraeasset.com/bbs...\n\n\n#중국#배당주#증시#밸류업 정책#수익률\n\n※ 보고서가 열리지 않는 경우, 다른 브라우저로 열어보시기 바랍니다.(인터넷 익스플로어, 크롬)\n\n스마트머니 구독 : https://bit.ly/3lluWtU\n챕터\n모두 보기\n하이라이트\n0:00\n중국 배당주 투자 열풍\n0:49\n투자대안의 부재와 높은 배당수익률\n1:37\n중국판 밸류업! 신국9조\n3:50\n스크립트\n스크립트를 보면서 시청하세요.\n스크립트 표시\n미래에셋 스마트머니\n구독자 149만명\n동영상\n정보\n페이스북\n[슬기로운 투자 생각] 중국의 전략적 투자 방향?\n제작자: 미래에셋 스마트머니\n조회수 2.1천회 · 3개월 전\n간략히"
0,슬기로운 투자 생각,2024. 6. 22.,"반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해"" #CATL | 슬기로운 투자 생각",댓글 6개,38,"조회수 7,797회",https://www.youtube.com/watch?v=Ms6p1HznC_U&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=3,"【이인구의 슬기로운 투자 생각】\nep.6 반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해""\n\n00:00 하이라이트\n00:31 주가 하락, 나는 진짜 억울!\n01:31 반등 시동! 글로벌 1위 배터리 업체\n02:18 한중 주요 배터리제조업체 수익성 비교\n02:53 CATL 주가 및 PER 밴드 \n03:31 클로징\n\n출연: 이인구 이사(미래에셋증권 디지털리서치팀)\n\n[슬기로운 투자 생각] ▼ 더보기\nep.1. 지금이 진짜 중국 증시 저점일까?\n • [슬기로운 투자 생각] 지금이 진짜 중국 증시 저점일까? \n\nep.2. 중국 증시 저점? 중국의 새로운 주도주는?\n • [슬기로운 투자 생각] 중국 증시 저점? 중국의 새로운 주도주는? \n\nep.3. 중국의 전략적 투자 방향?\n • [슬기로운 투자 생각] 중국의 전략적 투자 방향? \n\nep.4. 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이?\n • [슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는... \n\nep.5. 중국 고배당주 ETF, 투자 열풍 이유는?\n • [슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? ... \n\nep.6. 반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해""\n • 반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해"" #CATL ... \n\nep.7. 🌝중국 주도주 반등 랠리, 제대로 시동겁니다. 🚌 놓치기 전에 타세요~\n • 🌝중국 주도주 반등 랠리, 제대로 시동겁니다. 🚌 놓치기 전에 타세... \n\n\n[▼보고서 보기▼]\n[중국 투자 전략] 2024년, 중국 투자의 세가지 키워드\nhttps://securities.miraeasset.com/bbs...\n\n[중국 투자 전략] 2024년, 환승 차이나\nhttps://securities.miraeasset.com/bbs...\n챕터\n모두 보기\n하이라이트\n0:00\n주가 하락, 나는 진짜 억울!\n0:31\n반등 시동! 글로벌 1위 배터리 업체\n1:31\n한중 주요 배터리제조업체 수익성 비교\n2:18\n스크립트\n스크립트를 보면서 시청하세요.\n스크립트 표시\n미래에셋 스마트머니\n구독자 149만명\n동영상\n정보\n페이스북\n[슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? #Hangseng #중국주식 #고배당주\n제작자: 미래에셋 스마트머니\n조회수 5천회 · 1개월 전\n[슬기로운 투자 생각] 중국의 전략적 투자 방향?\n제작자: 미래에셋 스마트머니\n간략히"
0,슬기로운 투자 생각,2024. 6. 26.,"🌝중국 주도주 반등 랠리, 제대로 시동겁니다. 🚌 놓치기 전에 타세요~ [슬기로운 투자 생각] ep.7 #중국주식 #주도주",댓글 17개,278,"조회수 5,181회",https://www.youtube.com/watch?v=7Yoej1DUR_0&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=4,"【이인구의 슬기로운 투자 생각】\nep.6 반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해""\n\n00:00 하이라이트\n00:31 주가 하락, 나는 진짜 억울!\n01:31 반등 시동! 글로벌 1위 배터리 업체\n02:18 한중 주요 배터리제조업체 수익성 비교\n02:53 CATL 주가 및 PER 밴드 \n03:31 클로징\n\n출연: 이인구 이사(미래에셋증권 디지털리서치팀)\n\n[슬기로운 투자 생각] ▼ 더보기\nep.1. 지금이 진짜 중국 증시 저점일까?\n • [슬기로운 투자 생각] 

In [ ]:
youtube_data.reset_index(drop = True)[13:15]

,playlist,date,title,num_review,num_good,num_views,video_url,content
13,ETF로드맵,2024. 6. 20.,[ETF 로드맵🚩:기초편] Ep.10 ETF 주린이들의 궁금증① 해외 상장 ETF도 양도 소득세를? | 2024.06.20(목),댓글 1개,165,"조회수 3,569회",https://www.youtube.com/watch?v=ZUI6liW1Nnc&list=PL1-YXgQy7nWxZF1tsJhoZuCYKPPbZFrEc&index=2,"'ETF의 A부터 Z까지' \nETF를 알고 싶은 분들을 위해\n자세하고 친절하고 설명해드리겠습니다!😁\n\n🏃투자 가이드\n윤재홍 선임연구위원(미래에셋증권 디지털리서치팀)\n\n🚀빨리가기\n00:00 시작\n00:43 세금 고려 시 해외 상장 ETF vs 한국 상장 ETF\n02:55 해외 상장 ETF의 양도소득세\n05:17 초보자가 접근하기에 알맞은 ETF 종류, 접근 방법\n05:51 채권 ETF 투자 방법 및 전략\n07:05 환헤지형 ETF\n\n📖 교재보기\n[ETF 기초] 전 세계에 투자하는 1만 1,869가지 방법: 글로벌 ETF\nhttps://securities.miraeasset.com/bbs...\n\n[글로벌 채권형 ETF] 글로벌 채권형 ETF 트렌드, 주요 전략별 ETF 접근하기\nhttps://securities.miraeasset.com/bbs...\n\n[+α ETF] 달러는 부담스럽고 주식은 담고 싶을 때(2): 환헤지 ETF\nhttps://securities.miraeasset.com/bbs...\n\n✨AI요약\n해외 상장 ETF의 매매 차액 세율은 22% 양도 소득세가 부과됨.\n국내 상장 ETF와 해외 지수 추종 ETF의 세금 기준을 비교하여 설명.\n절세 계좌로는 ISA 계좌, 개인 연금 퇴직 연금 계좌 등이 있음.\n해외 상장 ETF는 양도 소득세와 배당 소득세가 발생함.\n증여를 통해 세금을 줄일 수 있는 방법에 대한 설명\n\n\n#ETF투자 #해외ETF #양도소득세 #절세전략 #금융소득종합과세 #증여세 #환전차액 #투자기초 #ETF초보 #세금정보\n\n* 본 영상은 미래에셋증권이 발행한 조사분석자료에 기반하였습니다.\n* 본 영상 및 조사분석자료의 지적재산권은 당사에 있으므로 당사의 허락 없이 무단 복제 및 배포할 수 없습니다.\n* 본 영상에서 사용된 조사분석자료는 미래에셋증권이 신뢰할 수 있는 자료 및 정보로부터 얻은 것이나, 당사가 그 정확성이나 완전성을 보장할 수 없으므로 투자자 자신의 판단과 책임하에 종목 선택이나 투자시기에 대한 최종 결정을 하시기 바랍니다. 따라서 본 영상과 조사분석자료는 어떠한 경우에도 고객의 증권투자 결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다.\n* 본 영상에 사용된 조사분석자료는 당사 홈페이지(securities.miraeasset.com)에서 열람 가능합니다.\n* 본 영상물은 투자정보 등 대외 제공에 관한 당사의 전자통신수단 관련 내부통제기준을 준수하고 있습니다.\n\n\n스마트머니 구독 : https://bit.ly/3lluWtU\n챕터\n모두 보기\n시작\n0:00\n세금 고려 시 해외 상장 ETF vs 한국 상장 ETF\n0:43\n해외 상장 ETF의 양도소득세\n2:55\n초보자가 접근하기에 알맞은 ETF 종류, 접근 방법\n5:17\n스크립트\n스크립트를 보면서 시청하세요.\n스크립트 표시\n미래에셋 스마트머니\n구독자 149만명\n동영상\n정보\n페이스북\n간략히"
14,ETF로드맵,2024. 4. 29.,"[ETF 로드맵🚩:기초편] Ep.09 ETF의 가격, 참고용어, 상장폐지,세금 등등 | 2024.04.29(월)","댓글 1,237개",1.9천,"조회수 26,140회",https://www.youtube.com/watch?v=dVqzMGg-zyQ&list=PL1-YXgQy7nWxZF1tsJhoZuCYKPPbZFrEc&index=3,"'ETF의 A부터 Z까지' \nETF를 알고 싶은 분들을 위해\n자세하고 친절하고 설명해드리겠습니다!😁\n\n🏃투자 가이드\n윤재홍 선임연구위원(미래에셋증권 디지털리서치팀)\n\n🚀빨리가기\n00:00 시작\n00:14 ETF의 가격\n01:20 참고용어\n02:40 ETF·ETN이름으로 성격 파악하기\n03:26 상장폐지\n04:26 세금\n05:10 클로징\n\n📖 교재보기\n[ETF 기초] 전 세계에 투자하는 1만 1,869가지 방법: 글로벌 ETF\nhttps://securities.miraeasset.com/bbs...\n\n* 본 영상은 미래에셋증권이 발행한 조사분석자료에 기반하였습니다.\n* 본 영상 및 조사분석자료의 지적재산권은 당사에 있으므로 당사의 허락 없이 무단 복제 및 배포할 수 없습니다.\n* 본 영상에서 사용된 조사분석자료는 미래에셋증권이 신뢰할 수 있는 자료 및 정보로부터 얻은 것이나, 당사가 그 정확성이나 완전성을 보장할 수 없으므로 투자자 자신의 판단과 책임하에 종목 선택이나 투자시기에 대한 최종 결정을 하시기 바랍니다. 따라서 본 영상과 조사분석자료는 어떠한 경우에도 고객의 증권투자 결과에 대한 법적 책임소재의 증빙자료로 사용될 수 없습니다.\n* 본 영상에 사용된 조사분석자료는 당사 홈페이지(securities.miraeasset.com)에서 열람 가능합니다.\n* 본 영상물은 투자정보 등 대외 제공에 관한 당사의 전자통신수단 관련 내부통제기준을 준수하고 있습니다.\n\n===================================================================\n🎁 이벤트🎁 \n\n[ETF 로드맵 : 기초편] Q&A 댓글 이벤트\n\n[ETF 로드맵 : 기초편]을 시청하시면서 궁금하셨던 내용을 댓글로 남겨주세요!\n\n안녕하세요, 구독자 여러분😄\n드디어 [ETF 로드맵 : 기초편] 9편이 모두 공개되었습니다~\n약 한 달에 걸쳐 ETF의 기초에 대해 학습해봤는데요. 유익한 시간이었나요~? \n\n부디 유익했기를 바라며, 마지막으로 여태까지 학습한 내용중 \n잘 모르겠다거나 더 자세히 알고 싶은 부분에 대해 질문 댓글을 남겨 주시기 바랍니다!\n\n👨‍🏫윤선생님이 여러분의 질문 댓글 중 10개를 선정하여 \nQ&A 영상을 통해 답변도 드리고, 선물도 드린다고 합니다!\n많은 참여 부탁드리며, 질문 댓글은 9화 영상에 남겨 주시면 됩니다.\n\n[ETF 로드맵:기초편] 9화에 질문 댓글을 남겨 주신 분들 중 \n윤선생님이 직접 선정한 10분께 ‘BBQ 황금올리브치킨+콜라 1.25L’ 쿠폰 1매를,\n추첨을 통해 총 100분께 ‘스타벅스 아이스 카페 아메리카노 T 2잔 + 부드러운 생크림 카스텔라’ 쿠폰 1매를 드립니다!\n\n▶ 댓글 선정 기준\n1. [ETF 로드맵 : 기초편] 1~9화 내용과 연관 있는 질문 중 꼭 필요한 내용\n2. 유사한 내용은 작성 시간 및 응모 시간 기준으로 판정\n\n▶ 이벤트 기간 및 발표\n기간 : 2024년 4월 29일(월) ~ 5월 5일(일)\n발표 : 2024년 5월 15일(수)\n경품지급 : 2024년 5월 15일(수) 15시\n \n▶ 이벤트 참여 방법\nStep1. ‘미래에셋 스마트머니’ 채널 구독 하기\n\nStep2. ‘[ETF 로드맵:기초편] 9화’ 시청하고 질문 댓글 달기\n • [ETF 로드맵🚩:기초편] Ep.09 ETF의 가격, 참고용어, 상... \n\nStep3. 이벤트 URL로 접속하여 응모하면 참여 완료\nhttps://docs.google.com/forms/d/e/1FA...\n\n\n미래에셋증권 준법감시인 심사필 제24-703호 (2024.04.29~2024.05.05)\n챕터\n모두 보기\n시작\n0:00\nETF의 가격\n0:14\n참고용어\n1:20\nETF·ETN이름으로 성격 파악하기\n2:40\n스크립트\n스크립트를 보면서 시청하세요.\n스크립트 표시\n미래에셋 스마트머니\n구독자 149만명\n동영상\n정보\n페이스북\n간략히"


In [ ]:
# 판다스 옵션 설정
pd.set_option('display.max_colwidth', None)
row = {'a' : 'ㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁ'}

pd.DataFrame([row]).a

0    ㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁㅁ
Name: a, dtype: object